# Visualize travel time to cities and ports in the year 2015 maps

This notebook shows how to read and visualize travel time to cities and ports maps. These maps represent a unique global representation of physical access to essential services offered by cities and ports. 

Each accessibility layer is a 30 arc-second resolution raster in WGS84 (coordinate reference system EPSG:4326) projection with a bounding box of 85°N, 180°E, 60°S and 180°W. The format is single band GeoTIFF, 16 bit unsigned integer with 65,535 as the nodata value. The pixel values represent the time in minutes from that pixel to the nearest settlement. Travel times are reported for all pixels classified as land or inland water areas.

The dataset is available at the figshare repository https://doi.org/10.6084/m9.figshare.7638134.v3
The dataset and the validation are fully described in a Nature Scientific Data Descriptor https://www.nature.com/articles/s41597-019-0265-5


## 1. Installing the required packages

In [ ]:
!pip install matplotlib
!pip install pillow
!pip install numpy
!pip install opencv-python



## 2. Import required libraries

In [ ]:
import requests
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np

## 3. Set dataset id and base URL

In [ ]:
id = '7638134'
api_url = "https://api.figshare.com/v2/"

## 4. Retrieve Dataset

In [ ]:
# Define functions that read and download files
def read_public_article(id, api_url):
    request_url = f"{api_url}articles/{id}"
    print("Sending request to:", api_url)
    response = requests.get(request_url)
    status_code = response.status_code
    print(f"Response status: {status_code}")

    if status_code == 200:
        jsonResponse = response.json()
        return jsonResponse
    else:
        print("Error")
        return None
        # for key, value in jsonResponse.items():
        #     print(key, ":", value)

def download_file(folder, file_name, download_url):
    print("Downloading ", file_name)
    response = requests.get(download_url)
    status_code = response.status_code
    if (status_code == 200):
        print(f"File {file_name} was retrieved successfully!")
        open(os.path.join(folder, file_name) , 'wb').write(response.content)
        print(f"File address: {os.path.join(folder, file_name)}")
    else:
        print(f"Could not donwload {file_name}.")
    pass

def download_article_files(folder, article_json):
       
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"{folder} directory was created!")

    for item in article_json['files']:       
        
        # file_title = file_name_dict[item['name']]
        # print(n)
        if item['name'].endswith('.tif'):
            download_file(folder, item['name'], item['download_url'])
        
        
    pass

file_name_dict = {
        'travel_time_to_cities_1.tif': 'Travel_Time_to_Cities_5000000_50000000',
        'travel_time_to_cities_2.tif': 'Travel_Time_to_Cities_1000000_5000000',
        'travel_time_to_cities_3.tif': 'ravel_Time_to_Cities_500000_1000000',
        'travel_time_to_cities_4.tif': 'Travel_Time_to_Cities_200000_500000',
        'travel_time_to_cities_5.tif': 'Travel_Time_to_Cities_100000_200000',    
        'travel_time_to_cities_6.tif': 'Travel_Time_to_Cities_50000_100000',
        'travel_time_to_cities_7.tif': 'Travel_Time_to_Cities_20000_50000',
        'travel_time_to_cities_8.tif': 'Travel_Time_to_Cities_10000_20000',
        'travel_time_to_cities_9.tif': 'Travel_Time_to_Cities_5000_10000',
        'travel_time_to_cities_10.tif': 'Travel_Time_to_Cities_20000_110000000',
        'travel_time_to_cities_11.tif': 'Travel_Time_to_Cities_50000_50000000',
        'travel_time_to_cities_12.tif': 'Travel_Time_to_Cities_5000_110000000',
        'travel_time_to_ports_1.tif': 'Travel_Time_to_ports_160',
        'travel_time_to_ports_2.tif': 'Travel_Time_to_ports_361',
        'travel_time_to_ports_3.tif': 'Travel_Time_to_ports_990',
        'travel_time_to_ports_4.tif': 'Travel_Time_to_ports_2153',
        'travel_time_to_ports_5.tif': 'Travel_Time_to_ports_3778'}
        
article_json = read_public_article(id=id, api_url=api_url)
download_article_files('data', article_json=article_json)

## 5. Show the travel time maps

In [ ]:
Image.MAX_IMAGE_PIXELS = None

def show_maps(folder, file_name_dict):
    n = 0
    file_name = list(file_name_dict.keys())
    for item in file_name_dict:
        n = n+1
        plt.figure(n)
        plt.imshow(Image.open(os.path.join(folder,file_name[n-1])))
        plt.axis('off')
        plt.title(file_name_dict[item])
    pass
show_maps('data', file_name_dict = file_name_dict)


## 6. Combination of different maps

The nine layers represent travel times to human settlements of different population ranges.
In the following code, change the minimum and maximum population you would like to create a new travel time map.

In [ ]:
Image.MAX_IMAGE_PIXELS = None
maps_dict = {
        'travel_time_to_cities_1.tif': 'Travel_Time_to_Cities_5000000_50000000',
        'travel_time_to_cities_2.tif': 'Travel_Time_to_Cities_1000000_5000000',
        'travel_time_to_cities_3.tif': 'ravel_Time_to_Cities_500000_1000000',
        'travel_time_to_cities_4.tif': 'Travel_Time_to_Cities_200000_500000',
        'travel_time_to_cities_5.tif': 'Travel_Time_to_Cities_100000_200000',    
        'travel_time_to_cities_6.tif': 'Travel_Time_to_Cities_50000_100000',
        'travel_time_to_cities_7.tif': 'Travel_Time_to_Cities_20000_50000',
        'travel_time_to_cities_8.tif': 'Travel_Time_to_Cities_10000_20000',
        'travel_time_to_cities_9.tif': 'Travel_Time_to_Cities_5000_10000',
        }
# input minimum and maximum population
Min_Population = input("choose the minimum population")
Max_Population = input("choose the maximum population")

# get the minimum of the travel time between the selected maps

def find_min_travel_time_map(Min_Population, Max_Population, maps_dict):

    # select relevant maps
    new_maps_dict ={}
    for item in maps_dict:
        x = maps_dict[item].split('_')
        if int(Min_Population) <= int(x[4]) < int(Max_Population):
            file_name = list(maps_dict.keys())[list(maps_dict.values()).index(maps_dict[item])]
            new_maps_dict[file_name] = maps_dict[item]
            
    maps_name = list(new_maps_dict.keys())
    
    # get the geoinformation of the maps
    def get_geoinfo(folder, file_name):
        sourceds = Image.open(os.path.join(folder,file_name))
        width, height = sourceds.size
        return width, height

    
    width, height = get_geoinfo('data', file_name = maps_name[0])
    
    # read relevant maps and creat a list of them
    maps_list = np.zeros((height,width,3,len(maps_name)), np.uint16)
    for i in range(len(maps_name)):
        maps_list[:,:,:, i] = cv2.imread(os.path.join('data',maps_name[i]))
    new_travel_time_map = maps_list.min(axis=3)
    return new_travel_time_map

new_travel_time_map = find_min_travel_time_map (Min_Population, Max_Population, maps_dict)

### 6.1 show the new travel time map

In [ ]:
plt.figure(1)
plt.imshow(new_travel_time_map)
plt.title('Travel_time_to_cities_' + Min_Population + '_' + Max_Population)

## 7. Create a subset of map

In [ ]:
#from osgeo import gdal

#upper_left_x = input("input the upper left for X")
#upper_left_y = input("input the upper left for Y")
#lower_right_x = input("input the lower right for X")
#lower_right_y = input("input the lower right for Y")
#window = (upper_left_x,upper_left_y,lower_right_x,lower_right_y)

#gdal.Translate('output_crop_map.tif', 'map.tif', projWin = window)